In [1]:
# import
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer

from konlpy.tag import Okt
from tqdm import tqdm
import pandas as pd


# local only
import os
import cx_Oracle as oci
os.environ["NLS_LANG"] = ".AL32UTF8"      #encodring이 UTF-8 인 경우

import re
import matplotlib.pyplot as plt

In [2]:
def listToString(s):  
    str1 = ""  
    for ele in s:  
        str1 += " " + ele.strip()  
    return str1

In [45]:
def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    pattern = '[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]'        # 특수기호제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    text = text.replace('\n', '')
    text = text.replace('\r', '')
    return text   

In [4]:
###############
# 1.데이터 불러오기
#####

# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_CRM.txt", 'r', encoding='utf8')
con_text = con_text.read()

def OutputTypeHandler(cursor, name, defaultType, size, precision, scale):
    if defaultType == oci.CLOB:
        return cursor.var(oci.LONG_STRING, arraysize = cursor.arraysize)
    elif defaultType == oci.BLOB:
        return cursor.var(oci.LONG_BINARY, arraysize = cursor.arraysize)

# DB 접속 : 실제 접속
conn = oci.connect(con_text, encoding='UTF-8', nencoding='UTF-8')
conn.outputtypehandler = OutputTypeHandler
cursor = conn.cursor()

In [5]:
# query run
query = '''
        select user_id
        , create_dt
        , title
        , content
        , 'maeili' as channel
        from maeili.MI_CUS_INQUIRY
        where answer_dt between ADD_MONTHS( TRUNC(sysdate,'MM'), -6) and ADD_MONTHS( LAST_DAY(sysdate), -1 )
        and answer_id in ('maeili_cs_01', 'maeili_cs_02', 'maeili_cs_03', 'maeili_cs_04')
        '''

df1 = pd.read_sql(query, con = conn )

In [6]:
# query run
query = '''
        select (select mmb_id from  mmbship.tm_mmb_info where unfy_mmb_no = o.reg_unfy_mmbno) as user_id
        , reg_dtm as create_dt
        , tite as title
        , CTT as content
        , 'Maeil Do' as channel
        from mmbship.TCH_CUST_OPNN O
        where O.reg_dtm between ADD_MONTHS( TRUNC(sysdate,'MM'), -6) and ADD_MONTHS( LAST_DAY(sysdate), -1 )
        and O.answ_regr_id in ('member' , 'maeili', 'maeildo', 'mpoint')
        '''
df2 = pd.read_sql(query, con = conn )

In [7]:
df = pd.concat([df1, df2]).reset_index(drop=True)

In [8]:
df.groupby('CHANNEL')['USER_ID'].count()

CHANNEL
Maeil Do     206
maeili      1521
Name: USER_ID, dtype: int64

In [9]:
documents =list(df.CONTENT)

In [46]:
documents_temp = []

for text in documents:
    documents_temp.append(clean_str(text))

In [47]:
documents_temp[:10]

['작년에 아기 등록 하고  당시 출산 예정일 21 02 28  유산되었고올해 쌍둥이 임신하여 11 12에 태어났어요아기 추가만 가능하고 수정이 안되는데 어떻게 해야 할까요 ',
 '이안 남 새봄여성병원 새봄조리원 혼합 매일앱솔루트명작 2fl 이든 남 새봄여성병원 새봄조리원 혼합 매일앱솔루트명작 2fl 쌍둥이구요  21년11월12일생입니다',
 '저번에 2주전 금요일 쯤에 상담했을때  셀렉스 썬화이버 우수후기 금일 중에 공지하신다고 했는데 기다려도 공지를 볼 수가 없어서요 확인부탁드립니다 ',
 '네이버로 가입했고  아이등록하여 폭 넓은 혜택을 받으려니 로그아웃 후 정회원으로 가입하라고 뜹니다 지시한 사항대로 진행하니 이미 가입되어있다고 뜹니다 탈퇴를 하고 재가입하려고해도 휴대전화인증을 하면 이미 가입되어있다고 뜹니다 해결방안 알려주세요',
 '안녕하세요 이번에 명작 2단계 리뉴얼된 것 체험단 선정이 되었는데요 제가 원래 버전의 명작과 같은줄 알고 아기가 이제 100일인데 2단계 체험단을 신청해서 당첨이 되었습니다   그런데 리뉴얼된 명작은 단계가 바뀌었더라구요   어떻게 해야할지 모르겠어서 문의 드립니다  이제 다음주면 100일 되는 아기인데 체험제품은 6개월부터 먹이는 제품이라   어떻게 해야할지 알려주세요 제대로 살펴보지 않고 신청해서 죄송합니다 ㅜㅜ',
 '둘째 아이 정보 수정 요청합니다출산일 2021 10 08이름 교현성별 남병원명 에덴병원조리원 에덴산후조리원수유상태 분유 엡솔루트1단계',
 '21년 4월10일생으로 수정부탁드립니다  ',
 '예비맘선물세트가 왔는데 로토토 역류방지쿠션  교환권이 안왔는데 이건어디서 받을수있을까용  ',
 '센서티브먹이면서 20ml스푼한번신청햇었는데요ㅜ이번에 명작으로바꾸었는데 명작20ml스푼신청할방법이없을까요 ㅠ',
 '첫재아기 날짜를 잘못등록해서 삭제하고십어요그리고 산양 모유50 할인쿠폰 받고십은데 자세하게 설명좀 해주세요 등록정보 계속하니 오류가떠요']

In [54]:
preprocessed_documents = []

for line in tqdm(documents_temp):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

100%|██████████████████████████████████████████████████████████████████████████| 1758/1758 [00:00<00:00, 387675.42it/s]


In [55]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [56]:
custom_tokenizer = CustomTokenizer(Okt())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=30000)
train_bow_embeddings = vectorizer.fit_transform(preprocessed_documents)

In [57]:
print(train_bow_embeddings.shape)

(1756, 5543)


In [58]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

In [59]:
id2token

{0: '00',
 1: '000',
 2: '000원',
 3: '01',
 4: '010',
 5: '01062515768',
 6: '01062647733',
 7: '01074337233',
 8: '01250',
 9: '01년',
 10: '01월',
 11: '02',
 12: '02월',
 13: '03',
 14: '04',
 15: '0413',
 16: '04월',
 17: '05',
 18: '0520',
 19: '05일',
 20: '06',
 21: '06월',
 22: '06일',
 23: '07',
 24: '070',
 25: '0725',
 26: '07월',
 27: '08',
 28: '080',
 29: '0808505539',
 30: '08월',
 31: '08일',
 32: '09',
 33: '09월',
 34: '09일',
 35: '0원',
 36: '10',
 37: '100',
 38: '1000',
 39: '1000원',
 40: '10020759008',
 41: '1008',
 42: '100원',
 43: '100일',
 44: '101',
 45: '1012',
 46: '1017',
 47: '102',
 48: '105',
 49: '1050',
 50: '106',
 51: '107',
 52: '109',
 53: '10시',
 54: '10월',
 55: '10일',
 56: '11',
 57: '11000원',
 58: '110928072612',
 59: '111007130338',
 60: '111118331105',
 61: '111124012500',
 62: '112',
 63: '1122',
 64: '1140',
 65: '114일',
 66: '116일',
 67: '11월',
 68: '11일',
 69: '12',
 70: '120',
 71: '1200',
 72: '1221010',
 73: '123',
 74: '124',
 75: '125',
 76: '12시'

In [69]:
vocab[:10]

['00',
 '000',
 '000원',
 '01',
 '010',
 '01062515768',
 '01062647733',
 '01074337233',
 '01250',
 '01년']

### LDA 학습하기

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [81]:
# TF-IDF 행렬로 변환
X = vectorizer.fit_transform(preprocessed_documents)

0it [15:27, ?it/s]


In [92]:
lda_model = LatentDirichletAllocation(n_components=20,learning_method='online',random_state=777,max_iter=1)
lda_top = lda_model.fit_transform(X)

In [93]:
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=7):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i]) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: ['충전', '잔액', '기프트카드', '취소', '정지', '0원', '한도']
Topic 2: ['선물', '세트', '유효하지', '통장', '예비', '승인', '어떻게']
Topic 3: ['생년', '월일', '21', '수정', '잘못', '생인데', '한자리']
Topic 4: ['생일', '수정', '2021', '변경', '다시', '15일', '산부인과']
Topic 5: ['오늘', '뜹니다', '해결', '혜택', '려고', '5년', '보니']
Topic 6: ['탈퇴', '후기', 'sns', '처리', '계정', '연동', '링크']
Topic 7: ['do', '바꿔서', '터져서', 'event', 'name', '왔어요', '들었습니다']
Topic 8: ['병원', '조리', '혼합', '수유', '않아', '여성', '검색']
Topic 9: ['기입', '7월', '11', '변경', '중복', '생일', '되어있어요']
Topic 10: ['회원', '가입', '이름', '주소', '으로', '개명', '변경']
Topic 11: ['교환', '요미', '하트', '10시', '월요일', '솔드아웃', '계속']
Topic 12: ['포인트', '적립', '쿠폰', '사용', '했는데', '확인', '번호']
Topic 13: ['아이', '정보', '등록', '수정', '아기', '어떻게', '신청']
Topic 14: ['단계', '응모', '임산부', '명작', '건지', '안보', '제품']
Topic 15: ['gt', 'lt', 'br', '주문', '철회', '중단', '배달']
Topic 16: ['당첨', '기프티콘', '이벤트', '문자', '발송', '주소', '참여']
Topic 17: ['자리', '숫자', '모르겠어요', '빠른', '하기를', '12', '수정']
Topic 18: ['당첨', '배송', '언제', '폴바', '이벤트', '연락', '문자']
Topic 19: 

In [ ]:
num_topics = 10      # 가설로 잡은 토픽의 갯수
chunksize = 2000     # 훈련 덩어리당 문서의 개수
passes = 10          # 학습 빈도, epochs과 같은 용어임, 최적화 결과수치를 사용함 
iterations = 400     # 각각 문서에 대해서 루프 횟수

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, id2word=dictionary, 
                                           num_topics = num_topics,
                                           chunksize = chunksize,
                                           passes=passes,
                                           iterations = iterations)

### KoBERT 학습하기

In [72]:
from bertopic import BERTopic

ModuleNotFoundError: No module named 'bertopic'

### Combined TM 학습하기

In [60]:
train_contextualized_embeddings = bert_embeddings_from_list(preprocessed_documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

In [61]:
qt = TopicModelDataPreparation()
training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

In [62]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=20, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [35120/35120]	Train Loss: 153.12029751232382	Time: 0:00:20.010113: : 20it [06:20, 19.03s/it]
Sampling: [20/20]: : 20it [05:09, 15.49s/it]


In [68]:
# 1줄로 나오는 게 토픽당 7개 키워드라서...
ctm.get_topics(7)

defaultdict(list,
            {0: ['242424', '인터넷', 'rkfka', '되고나서', '그렇지만', '시정', '심예린'],
             1: ['당첨', '쿠폰', '했는데', '신청', '다시', '배송', '가입'],
             2: ['해야하나요', '선물', '잘못', '하려는데', '나오네요', '됐다고', '어디서'],
             3: ['센서티브', '첨부', '있는데', '정보', '했는데', '신청', '사용'],
             4: ['부탁드려요', '없어서요', '21', '자꾸', '하려는데', '라고', '나오네요'],
             5: ['수정', '인데', '부탁드립니다', '스푼', '2021', '산후조리원', '루트'],
             6: ['등록', '혹시', '신청', '안녕하세요', '로그인', '체험', '후기'],
             7: ['일련번호', '하려는데', '라고', '는데', '2021년', '안되고', '가능할까'],
             8: ['안녕하세요', '문의', '분유', '아기', '단계', '으로', '등록'],
             9: ['늦지', '인터넷', '그렇지만', '242424', '이자', '기를', '이지만'],
             10: ['포인트', '언제', '장난감', 'ㅠㅠ', '결제', '싶은데', '적립'],
             11: ['분유', '하고', '등록', '아이', '입력', '까지', '이벤트'],
             12: ['최미화', '나가요', '심예린', 'rkfka', '기다려', '서울대', '242424'],
             13: ['안녕하세요', '병원', '루트', 'br', '쿠폰', '매일', '앱솔'],
             14: ['월일', '구입', '잘못', '안되는건', '날짜',

In [80]:
len(vocab)

5543

### ZeroShotTM 학습

In [73]:
from contextualized_topic_models.models.ctm import ZeroShotTM

In [74]:
ctm2 = ZeroShotTM(bow_size=len(vocab), contextual_size=768, n_components=20, num_epochs=20)
ctm2.fit(training_dataset) # run the model

Epoch: [20/20]	 Seen Samples: [35120/35120]	Train Loss: 152.7415927200491	Time: 0:00:40.026171: : 20it [12:54, 38.71s/it] 
0it [00:00, ?it/s]

RuntimeError: DataLoader worker (pid(s) 32020) exited unexpectedly

In [75]:
ctm2.get_topic_lists(5)

[['분유', '입력', '해서', '이벤트', '체험'],
 ['도전', '실렸었는데', '주정', '샤넬', '조차'],
 ['야한다고', '도전', '실렸었는데', '소사이어티', '이상해서요'],
 ['후기', '안녕하세요', '문의', '등록', '이벤트'],
 ['해볼려고', '의아', '입하면서', '있거든요', '뜨면서'],
 ['건가', '와서', '유산', '아까는', '나오네요'],
 ['아이', '문의', '입력', '센서티브', '으로'],
 ['접근', '5월', '됐는데', '오는', '개월수'],
 ['주정', '샤넬', '서비스', '이었는지', '여쭙니다'],
 ['어떻게', '따로', 'ㅠㅠ', '건가', '하는데'],
 ['안녕하세요', '해서', '당첨', '으로', '다시'],
 ['되어있어요', '중간', '오후', '잘못', '부탁'],
 ['실렸었는데', '주정', '야한다고', '있거든요', '조차'],
 ['다시', '가요', '답변', '주소', '안녕하세요'],
 ['으로', '문의', '아이', '하고', '쿠폰'],
 ['어제', '월요일', '어디', '하려는데', '건가'],
 ['그냥', '싶은데', '하트', '언제', '인가'],
 ['이름', '성별', '11', '산후조리원', '07'],
 ['요미', '으로', 'gt', 'lt', '하는'],
 ['째인데오늘', '보이네요', '이라입력해서', '되어있어서', '28일']]

In [77]:
ctm2

In [71]:
# import pyLDAvis as vis

# lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

# ctm_pd = vis.prepare(**lda_vis_data)
# vis.display(ctm_pd)